In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

In [2]:
def read_data(path):
    df = pd.read_csv(path, parse_dates=['FECHA'], infer_datetime_format=True)
    df['CPRECIO'] = df[' CPRECIO '].map(lambda x: x.strip().replace(",", ""))
    df['CPRECIO'] = df['CPRECIO'].convert_objects(convert_numeric=True)
    df['COSTOPESOS'] = df[' COSTOPESOS ']
    df = df.drop([' CPRECIO ', ' COSTOPESOS '], axis=1)
    cols = df.columns.values 
    cols[-3] = "YEAR"
    df.columns = cols
    return df

def read_test_data(path):
    df = pd.read_csv(path, parse_dates=['FECHA'], infer_datetime_format=True)
    df['CPRECIO'] = df[' CPRECIO ']
    df['COSTOPESOS'] = df[' COSTOPESOS ']
    df = df.drop([' CPRECIO ', ' COSTOPESOS '], axis=1)
    cols = df.columns.values 
    cols[-3] = "YEAR"
    df.columns = cols
    return df

def calculate_extra_cols(df):
    df['total_price'] =  df['CPRECIO'] * df['#UNIDADES'] * df['CTIPOCAM01']
    return df

df = read_data('./BASEVENTAS2010A2015.csv')
df = calculate_extra_cols(df)

# Cleanup all the spaces
df["MARCA"] = df["MARCA"].map(lambda x: x.strip())
df["IDPRODUCTO"] = df["IDPRODUCTO"].map(lambda x: x.strip())

/Users/Grimi/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [3]:
subdf = df[["FOLIO_FACTURA", "IDPRODUCTO", "#UNIDADES"]]
subdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154737 entries, 0 to 154736
Data columns (total 3 columns):
FOLIO_FACTURA    154737 non-null object
IDPRODUCTO       154737 non-null object
#UNIDADES        154737 non-null float64
dtypes: float64(1), object(2)
memory usage: 3.5+ MB


### Build Order to Product Matrix

In [4]:
order_prod = subdf.groupby(["FOLIO_FACTURA", "IDPRODUCTO"]).count().unstack()
order_prod = order_prod.fillna(0)
order_prod = order_prod["#UNIDADES"]

In [5]:
order_prod.shape

(19352, 15579)

### For product 25967 lets see which products are usually bought together

In [16]:
order_prod[order_prod["25967"] == 1].sum(axis=0).sort_values()[:-10:-1]

IDPRODUCTO
25967       527.0
62382       166.0
1668111      61.0
2966171      49.0
FAC_PROY     48.0
ANTICIPO     42.0
62985        42.0
1501252      41.0
1668124      41.0
dtype: float64

### We create the pairwise distance between products and similarity matrix

In [8]:
from sklearn.metrics.pairwise import pairwise_distances, cosine_similarity

similarities = cosine_similarity(order_prod.values.T)
distances = pairwise_distances(order_prod.values.T, metric="cosine")

In [9]:
# Confirm that we have the correct shape
print distances.shape, similarities.shape

(15579, 15579) (15579, 15579)


In [10]:
# For speed and ease lets create a lookup dictionary for item indices
item_dict = {}
item_arr = order_prod.columns
for idx, product in enumerate(order_prod.columns):
    item_dict[product] = idx

In [11]:
item_dict.items()[:10]

[('', 0),
 ('PF113AE', 14176),
 ('1SFA897108R7000', 3570),
 ('35549', 6158),
 ('73466', 7763),
 ('WLG2LDM1GJ03M', 15259),
 ('1SFA616921R2222', 3539),
 ('B8534M2-20', 8981),
 ('1SFA616921R2225', 3540),
 ('E2EQX7D1M1GJ', 9890)]

### Lets get some indices for the products commonly bought for item: 25967

In [12]:
print item_dict["25967"], item_dict["62382"], item_dict["1668111"], item_dict["2966171"]

4164 7112 2222 5085


### Now lets get the row for item: 25967 with index: 4164

In [13]:
distance = distances[4164]
similarity = similarities[4164]

### And test agains the indices that we found earlier

In [14]:
print distance[7112], distance[2222], distance[5085], item_arr[7112]
print similarity[7112], similarity[2222], similarity[5085], item_arr[7112]

0.527598695574 0.940160928459 0.952429831149 62382
0.472401304426 0.0598390715411 0.0475701688511 62382
